<a href="https://colab.research.google.com/github/virf96/Chat-Bot/blob/master/DatosFaltantes_SustitucionCategoriaAdicional_Sklearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sustitución usando una etiqueta adicional 'Missing' con Scikit-learn ==> SimpleImputer 

En la librería Scikit-learn hay una clase para manejar una gran variedad de métodos de sustitución.

El **SimpleImputer** es una clase que provee funcionalidad básica para la sustitución de valores ausentes, incluyendo:

- Sustitución por la media y la mediana para variables numéricas
- Sustitución por la categoría más frecuente para variables categóricas.
- Sustitución por valores arbitrarios para variables numéricas y categóricas.

### Ventajas

- Fácil de usar si se aplica a todo el dataframe
- Código mantenido por desarrolladores de Scikit-learn: buena calidad
- Rápida computación (usa NumPy para los cálculos)
- Permite usar grid-search (búsqueda en cuadrículas) para varios métodos de sustitución
- Permite usar diferentes valores para codificar ausencia de datos (se puede indicar si por ejemplo los valores nulos son np.nan, ceros, cadenas de caracteres vacías, u otros)

### Limitaciones

- Retorna un arreglo de NumPy en lugar de un dataframe de pandas, lo cual es inconveniente para el análisis de datos
- Necesita usar clases adicionales para seleccionar cuales variables se deben sustituir ==>
    - requiere líneas de código adicional
    - requiere ser usado con otras clases
    - no es tan sencillo de usar
    
### Más detalles acerca de los transformadores (transformers en inglés)

- [SimpleImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer)
- [ColumnTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html)
- [Stackoverflow](https://stackoverflow.com/questions/54160370/how-to-use-sklearn-column-transformer)


## En este demo:

Vamos a aprender **sustitución con una etiqueta adicional 'Missing' usando Scikit-learn** usando los datos Ames House Price.

- Para bajar los datos, por favor referirse a la clase  **Datasets** en la  **Sección 1** del curso.

### Nota: 
* 'Imputer' se deriva del verbo en inglés 'to impute' que quiere decir sustituir o reemplazar. Imputer es el objeto que completa la sustitución, de ahi el nombre dado a la clase.
* 'slicing' significa seleccionar conjuntos de datos (columnas/filas) de un ‘DataFrame’.
* 'Missing' -> ausente

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

# estas son las clases para sustitutición con sklearn
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# dividir dataset
from sklearn.model_selection import train_test_split

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# solo usaremos las siguientes variables categóricas en el demo:

# estas son las variables categóricas y el target SalePrice
cols_to_use = ['BsmtQual', 'FireplaceQu', 'SalePrice']

# carguemos los datos 
data = pd.read_csv('/content/drive/My Drive/datasets/houseprice.csv', usecols=cols_to_use)
data.head()

,BsmtQual,FireplaceQu,SalePrice
0,Gd,NaN,208500
1,Gd,TA,181500
2,Gd,TA,223500
3,TA,Gd,140000
4,Gd,TA,250000


In [11]:
# revisemos los valores nulos
data.isnull().mean().sort_values(ascending=False)

FireplaceQu    0.472603
BsmtQual       0.025342
SalePrice      0.000000
dtype: float64

Las variables cateogóricas BsmtQual y FirePlaceQu tienen datos ausentes


In [12]:
# separar datos en segmentos entrenamiento y prueba

# primero, separemos el target (SalePrice) del resto de las variables (features)
cols_to_use.remove('SalePrice')

X_train, X_test, y_train, y_test = train_test_split(data[cols_to_use], # solo las variables
                                                    data['SalePrice'], # el target
                                                    test_size=0.3, # el porcentaje de obs en el segmento de prueba
                                                    random_state=0) # para reproducir
X_train.shape, X_test.shape

((1022, 2), (438, 2))

In [13]:
# evaluemos el porcentaje de datos ausentes nuevamente
#Como podemos ver el porcentaje de valores ausentes cambio dado que solo estamos usando el 70% del set oiginal
#Recordemos que no consideramos a el target "SalePrice"
X_train.isnull().mean()

BsmtQual       0.023483
FireplaceQu    0.467710
dtype: float64

In [14]:
# exploremos los valores de la variable categórica
X_train['BsmtQual'].unique()

array(['Gd', 'TA', 'Fa', nan, 'Ex'], dtype=object)

In [15]:
# exploremos los valores de la variable categórica
X_train['FireplaceQu'].unique()

array([nan, 'Gd', 'TA', 'Fa', 'Po', 'Ex'], dtype=object)

In [16]:
# Ahora sustituyamos los valores faltantes con  SimpleImputer

# creemos una instancia de la clase SimpleImputer
# indicaremos que queremos sustituir los valores nulos
# con la categoría 'Missing'

imputer = SimpleImputer(strategy='constant', 
                       fill_value = 'Missing')

# ajustamos el imputer al segmento de entrenamiento
# en este caso simplemente reemplaza los valores nulos con el valor 'Missing'

imputer.fit(X_train)

SimpleImputer(add_indicator=False, copy=True, fill_value='Missing',
              missing_values=nan, strategy='constant', verbose=0)

In [17]:
# veamos los valores ajustados:
imputer.statistics_

array(['Missing', 'Missing'], dtype=object)

In [18]:
# ahora sustituyamos en los segmentos de entrenamiento y prueba

# NOTA: los datos son devueltos como un numpy array!!
X_train = imputer.transform(X_train)
X_test = imputer.transform(X_test)

X_train

array([['Gd', 'Missing'],
       ['Gd', 'Gd'],
       ['TA', 'Missing'],
       ...,
       ['Missing', 'Missing'],
       ['Gd', 'TA'],
       ['Gd', 'Missing']], dtype=object)

In [19]:
# transformemos el segmento de entrenamiento en un dataframe:

X_train = pd.DataFrame(X_train, columns=cols_to_use)
X_train.head()

,BsmtQual,FireplaceQu
0,Gd,Missing
1,Gd,Gd
2,TA,Missing
3,TA,Missing
4,TA,Missing


In [20]:
X_train['BsmtQual'].unique()

array(['Gd', 'TA', 'Fa', 'Missing', 'Ex'], dtype=object)

In [21]:
X_train.isnull().mean()

BsmtQual       0.0
FireplaceQu    0.0
dtype: float64

**ADVERTENCIA**:

Cuando usamos SimpleImputer y fijamos los parámetros:
- strategy='constant'
- fill_value = 'Missing'

Si el dataframe contiene variables que son numéricas y categóricas, los valores nulos NA en ambos serán reemplazados con 'Missing" y por lo tanto una variable numérica se convierte en categórica, que probablemente no es el efecto deseado.

La mayoría de datos contienen variables  numéricas y categóricas, por lo tanto lo más probable es que tendrás que usar un transformador para seleccionar las columnas como mostramos en los notebooks previos y en las siguientes celdas.


In [22]:
# carguemos los datos con variables numéricas y categóricas

cols_to_use = [
    'BsmtQual', 'FireplaceQu', 'LotFrontage', 'MasVnrArea', 'GarageYrBlt',
    'SalePrice'
]

data = pd.read_csv('/content/drive/My Drive/datasets/houseprice.csv', usecols=cols_to_use)
data.head()

,LotFrontage,MasVnrArea,BsmtQual,FireplaceQu,GarageYrBlt,SalePrice
0,65.0,196.0,Gd,NaN,2003.0,208500
1,80.0,0.0,Gd,TA,1976.0,181500
2,68.0,162.0,Gd,TA,2001.0,223500
3,60.0,0.0,TA,Gd,1998.0,140000
4,84.0,350.0,Gd,TA,2000.0,250000


In [23]:
# separar datos en segmentos entrenamiento y prueba

# primero descartemos el target de la lista de variables
cols_to_use.remove('SalePrice')

X_train, X_test, y_train, y_test = train_test_split(data[cols_to_use],
                                                    data['SalePrice'],
                                                    test_size=0.3,
                                                    random_state=0)
X_train.shape, X_test.shape

((1022, 5), (438, 5))

In [26]:
# revisemos los valores nulos
#El target "SalePrice" no fue considerado
X_train.isnull().mean().sort_values(ascending=False)

FireplaceQu    0.467710
LotFrontage    0.184932
GarageYrBlt    0.052838
BsmtQual       0.023483
MasVnrArea     0.004892
dtype: float64

En este demo, vamos a sustituir los valores nulos de las variables numéricas por la media y las variables categóricas por la nueva etiqueta 'Missing'.

In [28]:
# primero vamos a crear una lista, indicando cuales son las 
# variables a sustituir con cada método

features_numeric = ['LotFrontage', 'MasVnrArea', 'GarageYrBlt']
features_categoric = ['BsmtQual', 'FireplaceQu']

# luego vamos a instanciar imputers dentro de un pipeline
# creamos un imputer por cada variable
# indicando uno para la media y el otro para las variables categóricas

imputer_numeric = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
])

imputer_categoric = Pipeline(
    steps=[('imputer',
            SimpleImputer(strategy='constant', fill_value='Missing'))])

# luego ponemos las variables en lista y los transformadores juntos
# usando la columna transformer

preprocessor = ColumnTransformer(transformers=[('imputer_numeric',
                                                imputer_numeric,
                                                features_numeric),
                                               ('imputer_categoric',
                                                imputer_categoric,
                                                features_categoric)])

In [29]:
# ajustemos el preprocessor
preprocessor.fit(X_train)

ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
                  transformer_weights=None,
                  transformers=[('imputer_numeric',
                                 Pipeline(memory=None,
                                          steps=[('imputer',
                                                  SimpleImputer(add_indicator=False,
                                                                copy=True,
                                                                fill_value=None,
                                                                missing_values=nan,
                                                                strategy='mean',
                                                                verbose=0))],
                                          verbose=False),
                                 ['LotFrontage', 'MasVnrArea', 'GarageYrBlt']),
                                ('imputer_categoric',
                                 Pipeline

In [30]:
# podemos explorar el transformador:
preprocessor.transformers

[('imputer_numeric', Pipeline(memory=None,
           steps=[('imputer',
                   SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                                 missing_values=nan, strategy='mean',
                                 verbose=0))],
           verbose=False), ['LotFrontage', 'MasVnrArea', 'GarageYrBlt']),
 ('imputer_categoric', Pipeline(memory=None,
           steps=[('imputer',
                   SimpleImputer(add_indicator=False, copy=True,
                                 fill_value='Missing', missing_values=nan,
                                 strategy='constant', verbose=0))],
           verbose=False), ['BsmtQual', 'FireplaceQu'])]

In [31]:
# podemos ver los parámetros ajustados:

# para el imputer de las variables numéricas
preprocessor.named_transformers_['imputer_numeric'].named_steps['imputer'].statistics_

array([  69.66866747,  103.55358899, 1978.01239669])

In [32]:
# para el imputer de las variables categóricas
preprocessor.named_transformers_['imputer_categoric'].named_steps['imputer'].statistics_

array(['Missing', 'Missing'], dtype=object)

In [33]:
# y ahora podemos sustituir los segmentos de entrenamiento y prueba
# recuerda los datos retornados son numpy arrays

X_train = preprocessor.transform(X_train)
X_test = preprocessor.transform(X_test)

In [34]:
# ahora convirtamos el resultado en un dataframe
pd.DataFrame(X_train,
             columns=features_numeric+features_categoric).head()

,LotFrontage,MasVnrArea,GarageYrBlt,BsmtQual,FireplaceQu
0,69.6687,573,1998,Gd,Missing
1,69.6687,0,1996,Gd,Gd
2,50,0,1978.01,TA,Missing
3,60,0,1939,TA,Missing
4,60,0,1930,TA,Missing


In [35]:
# ahora convertimos el resultado en un dataframe
# y exploramos los valores ausentes
# (no debería haber ninguno)


X_train = pd.DataFrame(X_train,
             columns=features_numeric+features_categoric)

X_train.isnull().mean()

LotFrontage    0.0
MasVnrArea     0.0
GarageYrBlt    0.0
BsmtQual       0.0
FireplaceQu    0.0
dtype: float64